In [ ]:
import xarray as xr

### Boundary masks preparation

In [ ]:
grid_dest = 'fram_data/norfjords_160m_grid.nc'
grid_ds = xr.open_dataset(grid_dest)

In [ ]:
west_mask = grid_ds['mask_rho'].isel(xi_rho=0)
north_mask = grid_ds['mask_rho'].isel(eta_rho=-1)
east_mask = grid_ds['mask_rho'].isel(xi_rho=-1)
south_mask = grid_ds['mask_rho'].isel(eta_rho=0)

In [ ]:
bry_dest = 'fram_data/roho160_bry_GLORYS_20170115_to_20191130.nc'
bry_ds = xr.open_dataset(bry_dest)

In [ ]:
west_mask_array = xr.ones_like(bry_ds.temp_west) * west_mask
north_mask_array = xr.ones_like(bry_ds.temp_north) * north_mask
east_mask_array = xr.ones_like(bry_ds.temp_east) * east_mask
south_mask_array = xr.ones_like(bry_ds.temp_south) * south_mask

### Helper functions

In [ ]:
temp_tuple = ('temp_west', 'temp_north', 'temp_east', 'temp_south')
salt_tuple = ('salt_west', 'salt_north', 'salt_east', 'salt_south')

In [ ]:
def check_mask(parameter):
    """
    parameter: 'temperature' | 'salinity'
    """
    names = {'temperature': temp_tuple,
             'salinity': salt_tuple,
            }
    print(f"West: {(west_mask_array.astype(bool).values == bry_ds[names[parameter][0]].notnull().values).all()}")
    print(f"North: {(north_mask_array.astype(bool).values == bry_ds[names[parameter][1]].notnull().values).all()}")
    print(f"East: {(east_mask_array.astype(bool).values == bry_ds[names[parameter][2]].notnull().values).all()}")
    print(f"South: {(south_mask_array.astype(bool).values == bry_ds[names[parameter][3]].notnull().values).all()}")


In [ ]:
def backfill(parameter_tuple):
    """
    parameter_tuple: temp_tuple | salt_tuple
    """
    for name in parameter_tuple:
        bry_ds[name] = bry_ds[name].bfill('s_rho')
    print("Improper NaNs are replaced with values")


### Check temperature

In [ ]:
check_mask('temperature')
backfill(temp_tuple)
check_mask('temperature')

### Check salinity

In [ ]:
check_mask('salinity')
backfill(salt_tuple)
check_mask('salinity')

### Save the processed data

In [ ]:
bry_ds.to_netcdf("fram_data/roho160_bry.nc")